In [ ]:
import os
import json
import pickle

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from neuron import h
from dlutils import utils
from dlutils.cell import Cell, branch_order

The purpose of this notebook is to investigate the voltage attenuation in the dendrites in response to a depolarizing step of current in an individual with normal dendritic sodium and in a second one with exponentially decaying sodium.

In [ ]:
optimization_folder = '/Users/daniele/Postdoc/Research/Janelia/01_model_optimization/'

config_file = 'synaptic_inputs_active_a-thorny.json'
config = json.load(open(config_file, 'r'))

cell_type = config['cell_type']
prefix = cell_type[0].upper() + cell_type[1:]
base_folder = optimization_folder + prefix + '/' + config['cell_name'] + '/' + config['optimization_run'] + '/'
swc_file = config['swc_file']
cell_name = config['cell_name'] + '_'
individual = config['individual']
section_num = 7

swc_file = base_folder + swc_file
params_files = {'original': base_folder + 'individual_{}_orig.json'.format(individual),
               'modified': base_folder + 'individual_{}_exp_nax_apic.json'.format(individual)}
config_file = base_folder + 'parameters.json'

passive = False
with_TTX = False
replace_axon = True
add_axon_if_missing = True
parameters = {k:  json.load(open(v, 'r')) for k,v in params_files.items()}
mechanisms = utils.extract_mechanisms(config_file, cell_name)
sim_pars = pickle.load(open(base_folder + 'simulation_parameters.pkl','rb'))
replace_axon = sim_pars['replace_axon']
add_axon_if_missing = not sim_pars['no_add_axon']

In [ ]:
cells = {}
for k,pars in parameters.items():
    cells[k] = Cell('CA3_cell_%d' % int(np.random.uniform()*1e5), swc_file, pars, mechanisms)
    cells[k].instantiate(replace_axon, add_axon_if_missing, force_passive=passive, TTX=with_TTX)

In [ ]:
delay = 250
duration = 100
amplitude = 0.125
stim = {}
recorders = {}
for k,cell in cells.items():
    stim[k] = h.IClamp(cell.morpho.soma[0](0.5))
    stim[k].delay = delay
    stim[k].amp = amplitude
    stim[k].dur = duration
    recorders[k] = {'Vsoma': h.Vector(), 'Vdend': []}
    recorders[k]['Vsoma'].record(cell.morpho.soma[0](0.5)._ref_v)
    bo = {'soma': 0, 'dend': []}         # branch order
    dst = {'soma': 0, 'dend': []}         # distance from soma
    for sec in cell.morpho.apic:
        vec = h.Vector()
        vec.record(sec(0.5)._ref_v)
        recorders[k]['Vdend'].append(vec)
        bo['dend'].append(branch_order(sec))
        dst['dend'].append(h.distance(cell.morpho.soma[0](0.5), sec(0.5)))
    bo['dend'] = np.array(bo['dend'])
    dst['dend'] = np.array(dst['dend'])
    idx = np.argsort(dst['dend'])
    recorders[k]['Vdend'] = [recorders[k]['Vdend'][i] for i in idx]
    bo['dend'] = bo['dend'][idx]
    dst['dend'] = dst['dend'][idx]
time_recorder = h.Vector()
time_recorder.record(h._ref_t)

In [ ]:
h.cvode_active(1)
h.tstop = delay + duration + 100
h.run();

In [ ]:
time = np.array(time_recorder)
Vsoma = {k: np.array(rec['Vsoma']) for k,rec in recorders.items()}
Vdend = {k: np.array([rec for rec in recorder['Vdend']]) for k,recorder in recorders.items()}

In [ ]:
rows = bo['dend'].max() + 1
fig,ax = plt.subplots(rows, 2, sharex=True, sharey=True, figsize=(10, rows*2))
keys = ['original', 'modified']
idx, = np.where((time > delay+10) & (time < delay+45))
cmap_name = 'cividis'
cmap = plt.get_cmap(cmap_name, 1)
for i,k in enumerate(keys):
    ax[0,i].plot(time[idx], Vsoma[k][idx], color=cmap(0), lw=1)
    ax[-1,i].set_xlabel('Time (ms)')
    ax[0,i].set_title(k.capitalize())
ax[0,0].set_ylabel('Soma Vm (mV)')
for order in range(1, rows):
    jdx, = np.where(bo['dend'] == order)
    cmap = plt.get_cmap(cmap_name, len(jdx))
    for i,key in enumerate(keys):
        for j,k in enumerate(jdx):
            ax[order,i].plot(time[idx], Vdend[key][k,idx], color=cmap(j), lw=1, label='{:.0f} um'.format(dst['dend'][k]))
        ax[order,i].legend(loc='upper right')
    ax[order,0].set_ylabel('Vm @ B.O. {} (mV)'.format(order))
for side in ('right','top'):
    for i in range(rows):
        for j in range(2):
            ax[i][j].spines[side].set_visible(False)
fig.tight_layout()

Taken from Kim, S., Guzman, S. J., Hu, H., & Jonas, P. (2012). Active dendrites support efficient initiation of dendritic spikes in hippocampal CA3 pyramidal neurons. Nature Neuroscience, 15(4), 600–606. http://doi.org/10.1038/nn.3060

![Kim et al., 2012](Kim_2012.png)